<a href="https://colab.research.google.com/github/codefusser/AIAgents/blob/main/vision_agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vision Agents with smolagents


This notebook is part of the [Hugging Face Agents Course](https://www.hf.co/learn/agents-course), a free Course from beginner to expert, where you learn to build Agents.

![Agents course share](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/communication/share.png)

## Let's install the dependencies and login to our HF account to access the Inference API

If you haven't installed `smolagents` yet, you can do so by running the following command:

In [1]:
!pip install smolagents

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.3/105.3 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 48.0 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.


Let's also login to the Hugging Face Hub to have access to the Inference API.

In [3]:
from huggingface_hub import notebook_login

notebook_login()

## Providing Images at the Start of the Agent's Execution

In this approach, images are passed to the agent at the start and stored as `task_images` alongside the task prompt. The agent then processes these images throughout its execution.  

Consider the case where Alfred wants to verify the identities of the superheroes attending the party. He already has a dataset of images from previous parties with the names of the guests. Given a new visitor's image, the agent can compare it with the existing dataset and make a decision about letting them in.  

In this case, a guest is trying to enter, and Alfred suspects that this visitor might be The Joker impersonating Wonder Woman. Alfred needs to verify their identity to prevent anyone unwanted from entering.  

Let’s build the example. First, the images are loaded. In this case, we use images from Wikipedia to keep the example minimal, but image the possible use-case!

In [4]:
from PIL import Image
import requests
from io import BytesIO

image_urls = [
    "https://upload.wikimedia.org/wikipedia/commons/e/e8/The_Joker_at_Wax_Museum_Plus.jpg",
    "https://upload.wikimedia.org/wikipedia/en/9/98/Joker_%28DC_Comics_character%29.jpg"
]

images = []
for url in image_urls:
    response = requests.get(url)
    image = Image.open(BytesIO(response.content)).convert("RGB")
    images.append(image)

Now that we have the images, the agent will tell us wether the guests is actually a superhero (Wonder Woman) or a villian (The Joker).

In [5]:
print(images)

[<PIL.Image.Image image mode=RGB size=640x832 at 0x7AE3B4374C50>, <PIL.Image.Image image mode=RGB size=250x315 at 0x7AE3B4374890>]


In [7]:
!pip install -U google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 9.2 MB/s eta 0:00:00
  Attempting uninstall: google-genai
    Found existing installation: google-genai 1.8.0
    Uninstalling google-genai-1.8.0:
      Successfully uninstalled google-genai-1.8.0


In [8]:
from google import genai

In [9]:
from google.colab import userdata
import os
os.environ["GOOGLE_AI"] = userdata.get('GOOGLE_AI')

In [10]:
content_text = """
Describe the costume and makeup that the comic character in these photos is wearing and return the description.
Tell me if the guest is The Joker or Wonder Woman.
"""
content_image = images

In [17]:
from google import genai
from google.genai import types
client = genai.Client(api_key= os.environ['GOOGLE_AI'])
response = client.models.generate_content(
    model="gemini-2.0-flash-exp-image-generation",
    contents = [content_text, content_image],
    config=types.GenerateContentConfig(
        response_modalities = ['Text', 'Image']
        )
)

In [18]:
for part in response.candidates[0].content.parts:
  if part.text is not None:
    print(part.text)
  elif part.inline_data is not None:
    image = Image.open(BytesIO(part.inline_data.data))
    image.show()

The comic character in these photos is wearing a distinctive costume and makeup.

**Costume:**
*   **Purple Jacket:** They are wearing a purple suit jacket.
*   **Orange/Yellow Shirt:** Underneath, they have a bright orange or yellow collared shirt.
*   **Purple Tie/Scarf:** A loose, floppy purple tie or scarf is worn around the neck.

**Makeup:**
*   **White Face:** Their face is painted completely white.
*   **Red Lips:** They have exaggerated red lipstick, often forming a wide, sinister smile that extends beyond their natural lips.
*   **Dark Eye Makeup:** Dark makeup is applied around their eyes.

**The guest is The Joker.**


In [ ]:
from google.colab import userdata
import os
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [ ]:
from smolagents import CodeAgent, OpenAIServerModel

model = OpenAIServerModel(model_id="gpt-4o")

# Instantiate the agent
agent = CodeAgent(
    tools=[],
    model=model,
    max_steps=20,
    verbosity_level=2
)

response = agent.run(
    """
    Describe the costume and makeup that the comic character in these photos is wearing and return the description.
    Tell me if the guest is The Joker or Wonder Woman.
    """,
    images=images
)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Describe the costume and makeup that the comic character in these photos is wearing and return the description. │
│     Tell me if the guest is The Joker or Wonder Woman.                                                          │
│                                                                                                                 │
╰─ OpenAIServerModel - gpt-4o ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
I don't have the capability to identify or recognize people in images, but I can describe what I see.              
                                                                                                                   
The character in the photos you provided is wearing:                                                               
                                                                                                                   
1. **Costume:**                                                                                                    
   - A purple suit with a large bow tie in one image.                                                              
   - A white flower lapel and card in another image.                                                               
   - The style is flamboyant and colorful, typical of a comic villain.                                             
                                                                                                                   
2. **Makeup:**                                                                                                     
   - White face makeup covering the entire face.                                                                   
   - Red lips forming a wide, exaggerated smile.                                                                   
   - Dark makeup around the eyes.                                                                                  
   - Green hair.                                                                                                   
                                                                                                                   
From the description, this character resembles The Joker, a well-known comic book villain.                         

Error in code parsing:
Your code snippet is invalid, because the regex pattern ```(?:py|python)?\n(.*?)\n``` was not found in it.
Here is your code snippet:
I don't have the capability to identify or recognize people in images, but I can describe what I see.

The character in the photos you provided is wearing:

1. **Costume:**
   - A purple suit with a large bow tie in one image.
   - A white flower lapel and card in another image.
   - The style is flamboyant and colorful, typical of a comic villain.

2. **Makeup:**
   - White face makeup covering the entire face.
   - Red lips forming a wide, exaggerated smile.
   - Dark makeup around the eyes.
   - Green hair.

From the description, this character resembles The Joker, a well-known comic book villain.
Make sure to include code with the correct pattern, for instance:
Thoughts: Your thoughts
Code:
```py
# Your python code here
```<end_code>
Make sure to provide correct code blobs.

[Step 0: Duration 4.30 seconds| Input tokens: 3,004 | Output tokens: 139]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
I'm unable to identify characters in images, but I can offer a description.                                        
                                                                                                                   
Thought: From the images, I will describe the costume and makeup.                                                  
                                                                                                                   
Code:                                                                                                              
```py                                                                                                              
description = """                                                                                                  
1. Costume:                                                                                                        
   - A purple suit with a yellow shirt and a large purple bow tie.                                                 
   - Features a white flower lapel and a playing card in the second image.                                         
   - The style is flamboyant, consistent with a comic villain.                                                     
                                                                                                                   
2. Makeup:                                                                                                         
   - White face makeup covering the entire face.                                                                   
   - Red lips forming a wide, exaggerated smile.                                                                   
   - Blue eyeshadow with dark eye accents.                                                                         
   - Slicked-back green hair.                                                                                      
"""                                                                                                                
                                                                                                                   
# Based on the description, this character resembles The Joker.                                                    
character = "The Joker"                                                                                            
                                                                                                                   
final_answer({"description": description, "character": character})                                                 
```                                                                                                                

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  description = """                                                                                                
  1. Costume:                                                                                                      
     - A purple suit with a yellow shirt and a large purple bow tie.                                               
     - Features a white flower lapel and a playing card in the second image.                                       
     - The style is flamboyant, consistent with a comic villain.                                                   
                                                                                                                   
  2. Makeup:                                                                                                       
     - White face makeup covering the entire face.                                                                 
     - Red lips forming a wide, exaggerated smile.                                                                 
     - Blue eyeshadow with dark eye accents.                                                                       
     - Slicked-back green hair.                                                                                    
  """                                                                                                              
                                                                                                                   
  # Based on the description, this character resembles The Joker.                                                  
  character = "The Joker"                                                                                          
                                                                                                                   
  final_answer({"description": description, "character": character})                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: {'description': '\n1. Costume:\n   - A purple suit with a yellow shirt and a large purple bow 
tie.\n   - Features a white flower lapel and a playing card in the second image.\n   - The style is flamboyant, 
consistent with a comic villain.\n\n2. Makeup:\n   - White face makeup covering the entire face.\n   - Red lips 
forming a wide, exaggerated smile.\n   - Blue eyeshadow with dark eye accents.\n   - Slicked-back green hair.\n', 
'character': 'The Joker'}

[Step 1: Duration 7.36 seconds| Input tokens: 7,431 | Output tokens: 302]

In [ ]:
response

{'description': '\n1. Costume:\n   - A purple suit with a yellow shirt and a large purple bow tie.\n   - Features a white flower lapel and a playing card in the second image.\n   - The style is flamboyant, consistent with a comic villain.\n\n2. Makeup:\n   - White face makeup covering the entire face.\n   - Red lips forming a wide, exaggerated smile.\n   - Blue eyeshadow with dark eye accents.\n   - Slicked-back green hair.\n',
 'character': 'The Joker'}

In this case, the output reveals that the person is impersonating someone else, so we can prevent The Joker from entering the party!

## Providing Images with Dynamic Retrieval

This examples is provided as a `.py` file since it needs to be run locally since it'll browse the web. Go to the [Hugging Face Agents Course](https://www.hf.co/learn/agents-course) for more details.